In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


Suppose that we have a local constitutive law such that
$$
\varepsilon^{el}=M^{el}:\sigma
$$
for each individual grains.

In [2]:
inv=np.linalg.inv

-- Transformation rules for (2x1) matrix and (2x2) matrix.

In [3]:
def trans_2d_vec(a,r):
    b=np.zeros(2)
    for i in range(2):
        for j in range(2):
            b[i]=b[i]+r[i,j]*a[j]
    return b
def trans_2d_ten(a,r):
    b=np.zeros((2,2))
    for i in range(2):
        for j in range(2):
            for k in range(2):
                for l in range(2):
                    b[i,j]=b[i,j]+r[i,k]*r[j,l]*a[k,l]
    return b

In [4]:
ngr=3
weights=np.zeros(ngr)
weights[::]=1./ngr

In [5]:
thetas=[0,45,90]
thetas=np.array(thetas)*np.pi/180. ## to radian
R=np.zeros((2,2,len(thetas)))

for i in range(len(thetas)):
    r=np.zeros((2,2))
    r[0,0]=cos(thetas[i])
    r[0,1]=-sin(thetas[i])
    r[1,0]=sin(thetas[i])
    r[1,1]=cos(thetas[i])
    R[:,:,i]=r[:,:]

- Local elastic modulus

In [6]:
c=np.zeros((2,2))
c[0,0]=200000
c[1,1]=100000

In [7]:
print ('c:',c)

('c:', array([[200000.,      0.],
       [     0., 100000.]]))


## let's calculate the moduli in the lab axes coordinates

In [8]:
cgr=np.zeros((2,2,ngr))

In [9]:
for igr in range(ngr):
    r=R[:,:,igr]
    dum=trans_2d_ten(c,r)
    print('dum:',dum)
    cgr[:,:,igr]=dum

('dum:', array([[200000.,      0.],
       [     0., 100000.]]))
('dum:', array([[150000.,  50000.],
       [ 50000., 150000.]]))
('dum:', array([[1.000000e+05, 6.123234e-12],
       [6.123234e-12, 2.000000e+05]]))


In [10]:
print('cgr:',cgr.T)

('cgr:', array([[[2.000000e+05, 0.000000e+00],
        [0.000000e+00, 1.000000e+05]],

       [[1.500000e+05, 5.000000e+04],
        [5.000000e+04, 1.500000e+05]],

       [[1.000000e+05, 6.123234e-12],
        [6.123234e-12, 2.000000e+05]]]))


- weighted average of elastic stiffness

In [11]:
cav=np.zeros((2,2))
for igr in range(ngr):
    cav[:,:]=cav[:,:]+cgr[:,:,igr]*weights[igr]

In [12]:
print(cav/1000)

[[150.          16.66666667]
 [ 16.66666667 150.        ]]


In [15]:
inv(cav)

array([[ 6.75e-06, -7.50e-07],
       [-7.50e-07,  6.75e-06]])

* Extreme case 2.

In [21]:
sgr=cgr.copy()
for igr in range(ngr):
    dum=cgr[:,:,igr]
    sgr[:,:,igr]=inv(dum)

In [32]:
sav=np.zeros((2,2))
for igr in range(ngr):
    sav[:,:]=sav[:,:]+sgr[:,:,igr]*weights[igr]

In [34]:
print sav*100*12

[[ 0.009 -0.001]
 [-0.001  0.009]]
